# Start the Environment

In [19]:
import os
import sys
sys.path.append('D:/UserData/Z003XD5A/dev/deep_rl_projects/ml-agents/python')

from unityagents import UnityEnvironment
import numpy as np

In [20]:
env = UnityEnvironment(file_name='../../unity_environments/Tennis_Windows_x86_64/Tennis.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [21]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
print('states shape ', states.shape)
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
states shape  (2, 24)
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


# Agent in Action

In [22]:
import torch
import torch.nn.functional as F

import time
import numpy as np
from itertools import count
from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

from maddpg import *
from buffer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Hyperparameter settings

In [24]:
random_seed = 1234
gamma = 0.99
n_episodes = 2
tau = 2.e-1#1.e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ddpg_settings = {'state_size':state_size, 'action_size':action_size, 'random_seed':random_seed,
                 'hidden_in_dim':128, 'hidden_out_dim':128, 'activation':F.relu,'tau':tau, 'lr_actor':1e-3, 'lr_critic':1e-3,
                 'weight_decay':0., 'epsilon':1., 'epsilon_decay':0.}

maddpg = MADDPG(device, random_seed, gamma, ddpg_settings)

### Load Agent Parameters

In [25]:
for ia, ddpg in enumerate(maddpg.marl):
    actor_state_dict = torch.load('final_actor_%d.pth' % ia)
    critic_state_dict = torch.load('final_critic_%d.pth' % ia)
    ddpg.actor_local.load_state_dict(actor_state_dict)
    ddpg.critic_local.load_state_dict(critic_state_dict)
    

### Run

In [27]:
for i_episode in range(1, n_episodes+1):
        
    env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    maddpg.reset()

    score_average = 0
    start_time = time.time()
    while True:
        actions = maddpg.act(states, 0.)
        actions = np.hstack([action.numpy().squeeze() for action in actions]) # concatenate action [a11, a12, a21, a22] ij where i is agent index, j is action index
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        states = next_states                               # roll over states to next time step
        scores += rewards                                  # update the score (for each agent)            
        if np.any(dones):                                  # exit loop if episode finished
            break

In [28]:
env.close()